<center>
    <p align="center">
        <img src="https://logodownload.org/wp-content/uploads/2017/09/mackenzie-logo-3.png" style="height: 7ch;"><br>
        <h1 align="center">Computer Systems Undergradute Thesis</h1>
        <h2 align="center">Quantitative Analysis of the Impact of Image Pre-Processing on the Accuracy of Computer Vision Models Trained to Identify Dermatological Skin Diseases</a>
        <h4 align="center">Gabriel Mitelman Tkacz</a>
        </h4>
    </p>
</center>

<hr>

In [1]:
import tomllib
from functools import partial
from pprint import pprint
from typing import Sequence

import torch
import torch.nn as nn
import torch.optim as optim
from pynimbar import loading_animation
from torch.utils.data import DataLoader
from torchvision import transforms

from util import (
    BinaryCNN,
    LossFunction,
    NormalizeTransform,
    SkinDiseaseDataset,
    TestingDataset,
    TrainingDataset,
    ValidationDataset,
    evaluate,
    split_datasets,
)

In [2]:
with open("parameters.toml", "r") as f:
    parameters = tomllib.loads(f.read())

loading_handler = partial(
    loading_animation, break_on_error=True, verbose_errors=True, time_it_live=True
)

pprint(parameters)

{'TRAINING': {'batch_size': 128,
              'diseased_skin_path': './dataset/diseased/',
              'healthy_skin_path': './dataset/healthy/',
              'learning_rate': 0.0001,
              'num_epochs': 3,
              'num_workers': 12,
              'pin_memory': True,
              'precision_threshold': 0.8,
              'shuffle': True,
              'training_dataset_ratio': 0.8}}


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
training_ratio = parameters["TRAINING"]["training_dataset_ratio"]
testing_ratio = validation_ratio = round(1 - training_ratio, 1) / 2

print(f"Training ratio: {training_ratio*100}%")
print(f"Testing ratio: {testing_ratio*100}%")
print(f"Validation ratio: {validation_ratio*100}%")

seed = 47
print(f"\nSeed: {seed}")

Training ratio: 80.0%
Testing ratio: 10.0%
Validation ratio: 10.0%

Seed: 47


In [5]:
def get_model_data(
    to_transforms: Sequence[nn.Module] = list(),
) -> tuple[
    TrainingDataset,
    DataLoader,
    TestingDataset,
    DataLoader,
    ValidationDataset,
    DataLoader,
]:
    """
    This function returns the training and testing data loaders and datasets for the skin disease dataset.

    Args:
        to_transforms (Sequence[nn.Module], optional): A sequence of transforms to apply to the dataset. Defaullts to an empty sequence.

    Returns:
        dict[str, dict[str, DataLoader | SkinDiseaseDataset]]: A dictionary containing the training and testing data loaders and datasets.
    """
    base_transforms = [transforms.Resize((128, 128)), transforms.ToTensor()]
    transform = transforms.Compose([*base_transforms, *to_transforms])

    loader_kwargs = {
        "batch_size": parameters["TRAINING"]["batch_size"],
        "shuffle": parameters["TRAINING"]["shuffle"],
        "num_workers": parameters["TRAINING"]["num_workers"],
        "pin_memory": parameters["TRAINING"]["pin_memory"],
    }

    base_dataset = SkinDiseaseDataset(root_dir="dataset", transform=transform)
    train_dataset, test_dataset, validation_dataset = split_datasets(
        base_dataset, training_ratio, testing_ratio, validation_ratio, seed
    )

    train_loader = DataLoader(train_dataset, **loader_kwargs)
    test_loader = DataLoader(test_dataset, **loader_kwargs)
    validation_loader = DataLoader(validation_dataset, **loader_kwargs)

    return (
        train_dataset,
        train_loader,
        test_dataset,
        test_loader,
        validation_dataset,
        validation_loader,
    )


def evaluate_model(
    device: torch.device,
    train_loader: DataLoader,
    test_loader: DataLoader,
    validation_loader: DataLoader,
    criterion: LossFunction = nn.MSELoss(),
    optimizer_class: type[optim.Optimizer] = optim.Adam,
    learning_rate: float = parameters["TRAINING"]["learning_rate"],
) -> float:
    """
    This function evaluates the model using the given criterion and data loaders.

    Args:
        device (torch.device): The device to use for the evaluation.
        train_loader (DataLoader): The training data loader.
        test_loader (DataLoader): The testing data loader.
        validation_loader (DataLoader): The validation data loader.
        criterion (LossFunction): The loss function to use for the evaluation. Defaults to nn.BCELoss().
        optimizer_class (type[optim.Optimizer], optional): The optimizer class to use for the evaluation. Defaults to optim.Adagrad.

    Returns:
        float: The accuracy of the model.
    """
    criterion = criterion.to(device)

    model = BinaryCNN(device=device).to(device)
    optimizer = optimizer_class(model.parameters(), lr=learning_rate)  # type: ignore

    return evaluate(
        model=model,
        criterion=criterion,
        device=device,
        verbose=True,
        optimizer=optimizer,
        train_loader=train_loader,
        test_loader=test_loader,
        validation_loader=validation_loader,
        num_epochs=parameters["TRAINING"]["num_epochs"],
    )

## Class 0 Model: Images with no pre-processing

In [6]:
(
    base_train_dataset,
    base_train_loader,
    base_test_dataset,
    base_test_loader,
    base_validation_dataset,
    base_validation_loader,
) = get_model_data()

base_precision = evaluate_model(
    device, base_train_loader, base_test_loader, base_validation_loader
)

print(f"Base precision: {base_precision*100:.2f}%")

if base_precision < parameters["TRAINING"]["precision_threshold"]:
    raise ValueError("The base model did not meet the precision threshold.")

Using device: cuda
Epoch 1/3, Train Loss: 0.0807, Train Accuracy: 89.44%, Validation Loss: 0.2306, Validation Accuracy: 49.00%
Best model saved at epoch 1 with Validation Accuracy: 49.00%
Epoch 2/3, Train Loss: 0.0364, Train Accuracy: 95.31%, Validation Loss: 0.1139, Validation Accuracy: 87.00%
Best model saved at epoch 2 with Validation Accuracy: 87.00%
Epoch 3/3, Train Loss: 0.0294, Train Accuracy: 96.06%, Validation Loss: 0.0678, Validation Accuracy: 89.50%
Best model saved at epoch 3 with Validation Accuracy: 89.50%
Total training duration: 3.48 minutes
Test Accuracy of the Binary Classification Model: 89.50%
Base precision: 89.50%


## Class 1 Models: Images with only one pre-process

### Class 1.1 Models: Normalizing the image

In [7]:
(
    normalize_train_dataset,
    normalize_train_loader,
    normalize_test_dataset,
    normalize_test_loader,
    normalize_validation_dataset,
    normalize_validation_loader,
) = get_model_data([NormalizeTransform()])

normalize_precision = evaluate_model(
    device, normalize_train_loader, normalize_test_loader, normalize_validation_loader
)

normalize_precision_diff = normalize_precision - base_precision

print(f"Normalized precision: {normalize_precision*100:.2f}%")
print(
    f"That is an {'upgrade' if normalize_precision_diff > 0 else 'downgrade'} of {normalize_precision_diff*100:.2f}%."
)

Using device: cuda
Epoch 1/3, Train Loss: 0.0746, Train Accuracy: 90.88%, Validation Loss: 0.2002, Validation Accuracy: 66.00%
Best model saved at epoch 1 with Validation Accuracy: 66.00%
Epoch 2/3, Train Loss: 0.0312, Train Accuracy: 95.88%, Validation Loss: 0.0663, Validation Accuracy: 91.00%
Best model saved at epoch 2 with Validation Accuracy: 91.00%
Epoch 3/3, Train Loss: 0.0258, Train Accuracy: 96.50%, Validation Loss: 0.0466, Validation Accuracy: 93.50%
Best model saved at epoch 3 with Validation Accuracy: 93.50%
Total training duration: 3.43 minutes
Test Accuracy of the Binary Classification Model: 92.00%
Normalized precision: 92.00%
That is an upgrade of 0.025000000000000022.
